In [1]:
%env HF_ENDPOINT=https://hf-mirror.com
from KeepFIT.keepfit.modeling.model import KeepFITModel
from retriever import TextRetriever, MMRetriever
import torch


env: HF_ENDPOINT=https://hf-mirror.com


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mmRetriever=MMRetriever(device=device,dataset_path="/home/jqxu/Ragas/MM_feature_set")
textRetriever=TextRetriever(device=device,dataset_path="/home/jqxu/Ragas/text_datasets/dpr-txt/combined_books")
model = KeepFITModel(vision_type='resnet_v2', out_path='./output', from_checkpoint=True,
                        vision_pretrained=True,
                        weights_path=f'/home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth')
model.to(device)

Pretrained weights: IMAGENET1K_V2


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


100%|██████████| 6/6 [00:00<00:00, 372.89it/s]


Pretrained weights: IMAGENET1K_V2
load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


KeepFITModel(
  (vision_model): VisionModel(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): R

In [3]:
from extract.extractor import Extractor
from query_processor import QueryProcessor
import datasets
import numpy as np
import random
from PIL import Image
api_key = "REMOVED"
mmdataset = datasets.load_from_disk("/home/jqxu/Ragas/MM_feature_set")

keyWordExtractor=Extractor(model='gpt-3.5-turbo',api_key=api_key)
headers={
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
queryProcessor=QueryProcessor(headers)

In [5]:
import os
random_indices=random.sample(range(len(mmdataset)), 2)
mmSamples = mmdataset.select(indices=random_indices)
for sample in mmSamples:
    img_path = sample['image_path']
    scores, retrieved_examples = mmRetriever.retrieve(img_path, k=1)
    similar_img_path = retrieved_examples["image_path"][0]
    similar_description = retrieved_examples["description"][0]
    print("Image Description:",sample["description"])
    keywords=keyWordExtractor(similar_description)
    scores,retrieved_texts = textRetriever.retrieve(query=keywords, k=3,is_keywords=True)
    combined_texts = [f"{title}: {text}" for title, text in zip(retrieved_texts["title"],retrieved_texts["text"])]
    result=queryProcessor.process_query(image_path=img_path)
    #save the text to .txt file
    with open("text.txt","w") as f:
        f.write("\n".join(result))
    print("-" * 100)


Image Description: Papillitis. The disc shows hyperemia and edema as well as blurring margin.
{'id': 'chatcmpl-89DOa3ICoDL16CF2tr3lfJIgb3gsw', 'object': 'chat.completion', 'created': 1724033054, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Keyword: [papilitis, disc hyperemia, both eyes, left eye] \n\nExplanation:'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 361, 'completion_tokens': 21, 'total_tokens': 382}}
['papilitis', ' disc hyperemia', ' both eyes', ' left eye']
Sending to OpenAI...
{'id': 'chatcmpl-89D6HRJwWl2XH4pE061xemRzDSnVZ', 'object': 'chat.completion', 'created': 1724033098, 'model': 'gpt-4o-all', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Retinal Fundus Image Analysis Report\n\n#### 1. **Optic Disc:**\n   - **Appearance:** The optic disc appears slightly elevated with blurred margins, particularly on the nasal side. This suggests the presence of papilledema or optic disc edem